# 01 Brain Diagnostics
This notebook demonstrates the capabilities of the MacePotential (The Brain), including initialization, training (Head-Only), and uncertainty quantification.


In [ ]:
import sys
import os
import torch
import numpy as np
from ase import Atoms
from ase.build import bulk
import matplotlib.pyplot as plt
from mace.modules import ScaleShiftMACE
from mace.modules.blocks import RealAgnosticInteractionBlock
from e3nn import o3

# Add src to path to import MacePotential
# Assuming this notebook is in tutorials_and_uat/cycle_02/
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

from src.potentials.mace_impl import MacePotential


In [ ]:
def create_dummy_model(path):
    # Minimal parameters for a valid MACE model
    model_config = dict(
        r_max=4.0,
        num_bessel=3,
        num_polynomial_cutoff=3,
        max_ell=1,
        interaction_cls=RealAgnosticInteractionBlock,
        interaction_cls_first=RealAgnosticInteractionBlock,
        num_interactions=1,
        num_elements=2,
        hidden_irreps=o3.Irreps('8x0e'),
        MLP_irreps=o3.Irreps('8x0e'),
        atomic_energies=np.array([0.0, 0.0]),
        avg_num_neighbors=1.0,
        atomic_numbers=[1, 29], # H, Cu
        correlation=1,
        gate=torch.nn.functional.silu,
    )
    
    # Create the model wrapper
    model = ScaleShiftMACE(
        atomic_inter_scale=1.0,
        atomic_inter_shift=0.0,
        **model_config
    )
    
    # Save it
    torch.save(model, path)
    print(f"Saved dummy model to {path}")

# Create the model file
create_dummy_model('temp_foundation.model')


In [ ]:
# Initialize the potential with the dummy model
pot = MacePotential('temp_foundation.model')
print("MacePotential initialized successfully.")


In [ ]:
# Generate synthetic training data (perturbed Copper crystals)
training_data = []
np.random.seed(42)
for i in range(5):
    atoms = bulk('Cu', cubic=True)
    atoms.rattle(stdev=0.1, seed=i)
    training_data.append(atoms)
print(f"Generated {len(training_data)} training structures.")


In [ ]:
# Capture weights to verify freezing logic
# Accessing internal model structure - this depends on MACE architecture
backbone_weight_before = pot.model.interactions[0].linear.weight.detach().clone()
readout_weight_before = pot.model.readouts[-1].linear.weight.detach().clone()

print("Captured initial weights.")


In [ ]:
# Train the model (Head-Only)
print("Starting training...")
# We don't provide atomic_energies or labels here, MacePotential.train calculates loss against the atoms' own potential energy 
# But wait, the atoms generated above don't have energies attached!
# We need to attach dummy energies/forces to the training data for the loss function to work.
# MacePotential.train uses: ref_energies.append(a.get_potential_energy())

# Let's attach dummy target values
for atoms in training_data:
    # Dummy energy: -3.5 eV per atom roughly for Cu
    e = -3.5 * len(atoms) + np.random.normal(0, 0.1)
    f = np.random.uniform(-0.1, 0.1, size=(len(atoms), 3))
    
    # Use SinglePointCalculator to attach results
    from ase.calculators.singlepoint import SinglePointCalculator
    atoms.calc = SinglePointCalculator(atoms, energy=e, forces=f)

pot.train(training_data, energy_weight=1.0, forces_weight=1.0)
print("Training complete.")


In [ ]:
# Verify that backbone is frozen and readout is updated
backbone_weight_after = pot.model.interactions[0].linear.weight
readout_weight_after = pot.model.readouts[-1].linear.weight

# Backbone should be identical
if torch.allclose(backbone_weight_before, backbone_weight_after):
    print("SUCCESS: Backbone weights are frozen.")
else:
    print("FAILURE: Backbone weights changed!")

# Readout should change
if not torch.allclose(readout_weight_before, readout_weight_after):
    print("SUCCESS: Readout weights updated.")
else:
    print("WARNING: Readout weights did not change.")


In [ ]:
# Plot Uncertainty vs Distortion
distortion_levels = np.linspace(0, 0.5, 10)
avg_uncertainties = []

for d in distortion_levels:
    atoms = bulk('Cu', cubic=True)
    atoms.rattle(stdev=d, seed=42)
    # get_uncertainty returns array of scores per atom
    u = pot.get_uncertainty(atoms)
    avg_uncertainties.append(np.max(u))

plt.figure(figsize=(8, 5))
plt.plot(distortion_levels, avg_uncertainties, marker='o-')
plt.xlabel("Distortion Level (stdev)")
plt.ylabel("Max Normalized Uncertainty")
plt.title("Uncertainty vs. Distortion")
plt.grid(True)
plt.show()


In [ ]:
# Verify u_max normalization
print(f"Model u_max: {pot.u_max}")

# Check that training data scores are <= 1.0 (approx)
max_train_score = 0.0
for atoms in training_data:
    u = pot.get_uncertainty(atoms)
    max_train_score = max(max_train_score, np.max(u))

print(f"Max score on training set: {max_train_score}")

if max_train_score <= 1.001:
    print("SUCCESS: Training data uncertainty is properly normalized (<= 1.0).")
else:
    print(f"FAILURE: Training data uncertainty {max_train_score} > 1.0")
